# FCS Lab 3 - MD5, Rainbow Tables

* Name(s): James Raphael Tiovalen
* Student ID(s): 1004555

## Part I: Hashing Using MD5

* How does the length of the hash correspond to the input string?

The length of the hash is **always the same** (128 bits or 32 lowercase alphanumeric characters), irrespective of what the input string's length is. Some examples to serve as demonstrations:

In [33]:
!echo -n "" | md5sum

d41d8cd98f00b204e9800998ecf8427e  -


In [34]:
!echo -n "foobar" | md5sum

3858f62230ac3c915f300c664312c63f  -


In [35]:
!echo -n "Foundations of Cybersecurity" | md5sum

4a9d1841ca774cb27461b4dfebb00761  -


In [36]:
!echo -n "The FitnessGram™ Pacer Test is a multistage aerobic capacity test that progressively gets more difficult as it continues. The 20 meter pacer test will begin in 30 seconds. Line up at the start. The running speed starts slowly, but gets faster each minute after you hear this signal. [beep] A single lap should be completed each time you hear this sound. [ding] Remember to run in a straight line, and run as long as possible. The second time you fail to complete a lap before the sound, your test is over. The test will begin on the word start. On your mark, get ready, start!" | md5sum

50be91caa7c7185f8d989bb3fbadb2d8  -


In [37]:
!echo -n "Glasses are really versatile. First, you can have glasses-wearing girls take them off and suddenly become beautiful, or have girls wearing glasses flashing those cute grins, or have girls stealing the protagonist's glasses and putting them on like, \"Haha, got your glasses!\" That's just way too cute! Also, boys with glasses! I really like when their glasses have that suspicious looking gleam, and it's amazing how it can look really cool or just be a joke. I really like how it can fulfill all those abstract needs. Being able to switch up the styles and colors of glasses based on your mood is a lot of fun too! It's actually so much fun! You have those half rim glasses, or the thick frame glasses, everything! It's like you're enjoying all these kinds of glasses at a buffet. I really want Luna to try some on or Marine to try some on to replace her eyepatch. We really need glasses to become a thing in Hololive and start selling them for HOLOCOMI. Don't. You. Think. We. Really. Need. To. Officially. Give. Everyone. Glasses?" | md5sum

955d81541f415e61189b29f6c89db548  -


* Are there any visible correlations between the hash and the input string?

There are approximately no naively visible correlations on first glance, although much closer inspection and analysis done by cybersecurity experts might reveal some underlying intricate small correlations (as demonstrated [here](https://www.links.org/?p=6)). While we ideally would like exactly zero correlation and no dependency at all between input strings and hashes, it is really difficult in real life to develop/invent/obtain such a perfect hash function. That said, existing hash functions are good enough, at least for the time being.

* What are the issues related to the cryptographic weakness of MD5?

MD5 is vulnerable to the "chosen-prefix collision attack", which allows attackers to generate hash collisions fairly easily/trivially by today's standards of computing power.

## Part II: Break Hashes with Brute Force

* How much time did you take in total?

We select the following constraints that we were given as the potential choice of characters for the 5-characters plaintext:
- `string.ascii_lowercase`
- `string.digits`

As such, using our brute force method, the time taken to crack/reverse all 15 hashes is around **655.34** seconds.

* How much time does it take to crack each string, on average?

It takes about 655.34 / 15 = **43.69** seconds to crack each string on average.

* Is it possible to amortize (gradually write off the initial cost of) the brute forcing attempts?

Yes, it is possible. This can be done either via forming lookup tables or by creating rainbow tables to store the initial hashes and brute forcing attempts.

## Part III: Creating Rainbow Tables

* Is rcrack faster/slower than your script ex2.py? By how much faster/slower is it?

`rcrack` is faster than my `ex2.py` script by around 655.34 - 2.54 = **652.8** seconds. Evidence shown below.

Obviously, execution time of `rcrack` would be much faster since it is evident that most of the time was spent generating the rainbow tables instead (`rtgen` took around 29.2 seconds to generate a rainbow table of size 9600000 bytes). This is because the runtime of a hash table lookup is much lower than generating said hash table. We can also see that `rcrack` spent most of the time in chain traversal (to do the lookup).

In [38]:
!cd ./rainbowcrack-1.8-linux64/ && ./rtgen md5 loweralpha-numeric 5 5 0 3800 600000 0
!cd ./rainbowcrack-1.8-linux64/ && ./rtsort .
!cd ./rainbowcrack-1.8-linux64/ && ./rcrack . -l ../hash5.txt

rainbow table md5_loweralpha-numeric#5-5_0_3800x600000_0.rt parameters
hash algorithm:         md5
hash length:            16
charset name:           loweralpha-numeric
charset data:           abcdefghijklmnopqrstuvwxyz0123456789
charset data in hex:    61 62 63 64 65 66 67 68 69 6a 6b 6c 6d 6e 6f 70 71 72 73 74 75 76 77 78 79 7a 30 31 32 33 34 35 36 37 38 39 
charset length:         36
plaintext length range: 5 - 5
reduce offset:          0x00000000
plaintext total:        60466176

sequential starting point begin from 0 (0x0000000000000000)
generating...
524288 of 600000 rainbow chains generated (0 m 25.5 s)
600000 of 600000 rainbow chains generated (0 m 3.7 s)
./md5_loweralpha-numeric#5-5_0_3800x600000_0.rt:
7943958528 bytes memory available
loading data...
sorting data...
writing sorted data...

1 rainbow tables found
memory available: 6339808460 bytes
memory for rainbow chain traverse: 60800 bytes per hash, 912000 bytes for 15 hashes
memory for rainbow table buffer: 2 x 9600016 by

* Do you observe any advantages or disadvantages of using rainbowcrack?

Using `rainbowcrack` (and rainbow tables in general) is advantageous when we are cracking multiple hashes. This is because it can be used to repeatedly attack other plaintexts with potentially a much faster speed than dictionary or bruteforce attacks. A meet-in-the-middle approach is also employed, allowing us to utilize some space to store some of the precomputed values and enabling us to achieve faster cracking time.

However, when we are cracking only a single hash, there is no point in using `rainbowcrack` (or rainbow tables in general) since the upper bound of the time complexity would be the same as bruteforce (potentially even worse in practical real-life applications since we need to compute the entire table regardless of the desired plaintext/hash's position in the chains). Another possible disadvantage would be in the case whereby we are low on space or memory capacity. In such a scenario, it would not be advisable to use `rainbowcrack` or rainbow tables.

## Part IV: Salt

* What is the observed differences between your ease of cracking the salted vs the unsalted plaintexts?

For salted plaintexts, we are not able to amortize the brute forcing attempts since we cannot "remove the salt" from the salted plaintexts. As such, it is more difficult to crack the salted plaintexts as compared to the unsalted plaintexts.

* Report the difference in time observed to crack.

Time taken to crack the salted hashes is significantly longer as compared to the time taken to crack the unsalted hashes.

As shown below, comparing to the attempt with same parameters to serve as a basis of comparison, we can clearly see that while `rtgen` takes around the same time to generate the rainbow tables (about 29.0 seconds) as for the unsalted case, the time taken to crack the salted hashes significantly increases to 27.34 seconds, and not all salted hashes were cracked by `rcrack`.

In [39]:
# Use same parameters
!cd ./rainbowcrack-1.8-linux64/ && ./rtgen md5 loweralpha-numeric 6 6 0 3800 600000 0
!cd ./rainbowcrack-1.8-linux64/ && ./rtsort .
!cd ./rainbowcrack-1.8-linux64/ && ./rcrack . -l ../salted6.txt

rainbow table md5_loweralpha-numeric#6-6_0_3800x600000_0.rt parameters
hash algorithm:         md5
hash length:            16
charset name:           loweralpha-numeric
charset data:           abcdefghijklmnopqrstuvwxyz0123456789
charset data in hex:    61 62 63 64 65 66 67 68 69 6a 6b 6c 6d 6e 6f 70 71 72 73 74 75 76 77 78 79 7a 30 31 32 33 34 35 36 37 38 39 
charset length:         36
plaintext length range: 6 - 6
reduce offset:          0x00000000
plaintext total:        2176782336

sequential starting point begin from 0 (0x0000000000000000)
generating...
524288 of 600000 rainbow chains generated (0 m 25.4 s)
600000 of 600000 rainbow chains generated (0 m 3.6 s)
./md5_loweralpha-numeric#5-5_0_3800x600000_0.rt:
8019419136 bytes memory available
loading data...
sorting data...
writing sorted data...

./md5_loweralpha-numeric#6-6_0_3800x600000_0.rt:
8017801216 bytes memory available
loading data...
sorting data...
writing sorted data...

2 rainbow tables found
memory available: 6416118

In [53]:
# Use custom parameters (by inspection, exploration, and trial-and-error)
# We use the same reduction function (same `table_index`) but we keep generating multiple consecutive chains (`part_index`) sequentially until we can crack all 15 hashes (to avoid needing to generate all 26 parts)
!cd ./rainbowcrack-1.8-linux64/ && for i in {1..10}; do ./rtgen md5 loweralpha-numeric 6 6 0 3800 600000 $i; done
!cd ./rainbowcrack-1.8-linux64/ && ./rtsort .
!cd ./rainbowcrack-1.8-linux64/ && ./rcrack . -l ../salted6.txt

rainbow table md5_loweralpha-numeric#6-6_0_3800x600000_1.rt parameters
hash algorithm:         md5
hash length:            16
charset name:           loweralpha-numeric
charset data:           abcdefghijklmnopqrstuvwxyz0123456789
charset data in hex:    61 62 63 64 65 66 67 68 69 6a 6b 6c 6d 6e 6f 70 71 72 73 74 75 76 77 78 79 7a 30 31 32 33 34 35 36 37 38 39 
charset length:         36
plaintext length range: 6 - 6
reduce offset:          0x00000000
plaintext total:        2176782336

sequential starting point begin from 600000 (0x00000000000927c0)
generating...
524288 of 600000 rainbow chains generated (0 m 26.1 s)
600000 of 600000 rainbow chains generated (0 m 4.1 s)
rainbow table md5_loweralpha-numeric#6-6_0_3800x600000_2.rt parameters
hash algorithm:         md5
hash length:            16
charset name:           loweralpha-numeric
charset data:           abcdefghijklmnopqrstuvwxyz0123456789
charset data in hex:    61 62 63 64 65 66 67 68 69 6a 6b 6c 6d 6e 6f 70 71 72 73 74 75 76 7

* Explain any differences between salted and non salted rcrack strategies.

Salted hashes require the computation of rainbow tables for **each** unique salt. This increase in the input space of possible hash-to-password pairs is why it takes a longer time to crack salted hashes (salted rcrack strategies) as compared to non-salted hashes (non-salted rcrack strategies).

There is quite a drastic discrepancy in performance with even the addition of a one character salt. This shows the importance of salting. The purpose of salting is to deter rainbow attacks, given its ability to (variably) increase the original plaintext's length. Given the increase in possible input space by one character, an attacker would have to scale up their rainbow tables significantly. Even if such an attacker knows the information that only a 1-character salt was added (as well knowing that it is a lowercase letter), the attacker would still need to deal with a 26-fold multiplicative increase in the space of possible input plaintexts.

In our case, while it is still technically feasible to compute all the rainbow tables for each salt since the salt's length is only 1 (one lowercase character), we would need to increase our used memory/storage space by 26 to store the 26-fold larger rainbow tables. Such amount of space might not be a luxury anymore, especially in lower end hardware systems. If we generate long salts uniquely attached to each plaintext, it would take a ridiculously large amount of time to compute **all** rainbow tables for **all** salts. As such, salting renders rainbow tables useless since storing extremely massive rainbow tables is impossible in practice, nullifying any possible tradeoff between space and computational time/effort.

## Part V: Hash Breaking Competition

* What is the approach you used to crack the hashes?

For the competition, we utilized a myriad of tools and methods. The approach that I used to crack the hashes is twofold:

1. Using dictionary and bruteforce attacks via [Hashcat](https://hashcat.net/hashcat/) and [John the Ripper](https://www.openwall.com/john/)

The "weak" hashes can be cracked via dictionary attacks fairly easily and quickly, while the "strong" hashes can be cracked via pure bruteforce attacks. For dictionary attacks, we utilize the [`OneRuleToRuleThemAll`](https://github.com/stealthsploit/Optimised-hashcat-Rule) rule or the [`pantagrule.one.royce`](https://github.com/rarecoil/pantagrule) rule, [CrackStation's main word list dictionary](https://crackstation.net/crackstation-wordlist-password-cracking-dictionary.htm), and [princeprocessor](https://github.com/hashcat/princeprocessor). For bruteforce attacks, we simply use incremental mask attacks using custom user-defined character sets (lowercase and uppercase alphanumerics with the more popularly used symbols).

2. Using online databases

The tricky hashes are the "moderate" hashes. They are not so easily cracked via pure dictionary attacks, but they often do not require pure bruteforce attacks (some even have fairly long lengths that would take virtually forever to crack via pure bruteforce, making brute force unfeasible). Instead, they require "smart" methods of thinking and cracking. One possible method would be to comb through online databases to get the plaintexts of these hashes. Such websites include:
  - [https://hashes.com/en/decrypt/hash](https://hashes.com/en/decrypt/hash)
  - [https://md5decrypt.net/en/](https://md5decrypt.net/en/)
  - [https://md5.gromweb.com/](https://md5.gromweb.com/)
  - [http://md5.my-addr.com/](http://md5.my-addr.com/)
  - [https://hashtoolkit.com/decrypt-md5-hash/](https://hashtoolkit.com/decrypt-md5-hash/)
  - [https://crackstation.net/](https://crackstation.net/)
  - [https://www.cmd5.org/](https://www.cmd5.org/)
  - [https://10015.io/tools/md5-encrypt-decrypt](https://10015.io/tools/md5-encrypt-decrypt)
  - [https://nitrxgen.net/md5db/](https://nitrxgen.net/md5db/)
  - [https://www.md5online.org/md5-decrypt.html](https://www.md5online.org/md5-decrypt.html)

This leaves us with **145** out of the **148** total hashes cracked.

To crack the remaining 3 hashes, we could potentially make use of the fact that they are considered as "moderate" hashes, and yet no online databases have their corresponding plaintexts. From this information, we could potentially deduce that the possible input space ideas might include:
- Aphorisms, short quotes, or popular sayings
- Phone number syntaxes
- Repeated characters (potentially in the `string.printable` charset) for very long lengths
- Double/triple hashes of common passwords (checks for potential fixed points have been conducted to no avail)
- Strings of English words with/without whitespaces/hyphens/slashes/underscores (there are approximately about 170000-520000 meaningfully distinct English words in existence)
- Somewhat obvious URL/URI syntaxes or website names
- Hybrid format (English word/words in leetspeak appended/prepended with random characters) - this would require combination attacks
- Popular song lyrics
- Titles/notes of popular musical pieces (like 4'33", etc.)
- Radio station format (140.45FM)
- Names of popular people
- Morse code
- Decimal representation of popular constants (speed of light, natural number, etc.), or simply a random string of digits with a comma/decimal point somewhere
- First sentence or even the entirety of popular literary texts (like one of Shakespeare's works or Lorem Ipsum) which might include different cases, punctuations and whitespaces - this would be literally impossible to brute force even with all the time in the universe and the computational power of all of humanity due to the huge size of the input space associated with it
- Anything related to SUTD 50.042 or [MIT 6.857](https://courses.csail.mit.edu/6.857/2014/files/hashes.txt) (might be useful to create custom word lists/dictionaries with custom ruleset files for this).

The nature of some of these hashes might not even be suitable for practical, real-life "passwords" in real-world scenarios, as shown by the more "creative" cracked plaintexts. As such, due to this aspect of the competition, it can be relatively harder to crack some of said hashes (especially those in the Moderate category), as the possible input space size greatly increases due to the sheer number of potential variations.

* How you decided or designed your approach?

This approach was designed after running Hashcat on the list of hashes for a while and noticing that the hashes in each category are of a particular "type". They are not ordered in terms of difficulty, but rather, ordered in terms of what type of approach we should take in cracking them. Rainbow tables are useless in this challenge since the input string's length and character set can vary tremendously, increasing the input space, and each hash might have different dynamic salt values (or even no static salts at all).

* What are the main challenges and limitations of your approach?

My approach is a complex approach which involves a lot of both automated and manual work. It can be challenging to manage and split the work properly, especially considering the fact that we should avoid wastage of effort.

My approach also does not guarantee that we will obtain the plaintexts of all possible MD5 hashes, since we employ a multi-targeted strategy with various "intuitive"/vaguely-defined stopping conditions, that might actually prevent us from getting the very plaintexts that we are attempting to get all along.

The exploratory nature of my approach might also eliminate or reduce the possibility of myself attempting more systematic approaches (which admittedly, might take a much longer time to crack all the hashes). It is also more difficult to document every single step in detail.